In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# 모델 만들기

In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
import warnings
warnings.filterwarnings(action='ignore')

In [ ]:
# 데이터 로드
dataset = pd.read_csv('/kaggle/input/pima-indians-diabetes-database/diabetes.csv', delimiter=',')
# dataset = np.loadtxt('/kaggle/input/pima-indians-diabetes-database/diabetes.csv', skiprows=1, delimiter=',')
X = dataset.iloc[:, :8]
y = dataset.iloc[:, 8]

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.1, random_state = 7)

In [ ]:
# 모델 학습 시키기
model = XGBClassifier()
model.fit(x_train, y_train)

# 예측하기
y_pred = model.predict(x_test)
predictions = [round(value) for value in y_pred]

# 평가하기
acc = accuracy_score(y_test, predictions)
print(f"Acc is {acc*100}")

In [ ]:
# 예제 4.2 특정 환자에 대해 당뇨병을 진단하는 코드
value = np.array([[1, 161, 72, 35, 0, 28.1, 0.527, 20]])
l = model.predict_proba(value)
print(f"No diabetes: {l[0][0]}, YES is {l[0][1]}")

# 의사결정 트리 시각화

In [ ]:
%matplotlib inline
import os
from xgboost import plot_tree
import matplotlib.pyplot as plt
from matplotlib.pylab import rcParams
rcParams['figure.figsize']=200,200

In [ ]:
plot_tree(model)
plt.show()

In [ ]:
# 깊이 조절하기
model_2 = XGBClassifier(max_depth=2)
model_2.fit(x_train, y_train)

plot_tree(model_2)
plt.show()

In [ ]:
model_4 = XGBClassifier(max_depth=4)
model_4.fit(x_train, y_train)

In [ ]:
rcParams['figure.figsize']=40,40
plot_tree(model_4)
plt.show()

## 깊이에 따른 모델 정확도 평가


In [ ]:
# 예측하기
y_pred = model_2.predict(x_test)
predictions = [round(value) for value in y_pred]
acc = accuracy_score(y_test, predictions)
print(f"model_2 is {acc*100}")

# 예측하기
y_pred = model_4.predict(x_test)
predictions = [round(value) for value in y_pred]
acc = accuracy_score(y_test, predictions)
print(f"model_4 is {acc*100}")

* depth=2인 모델이 75%의 정확도를 가짐.
* depth=4인 경우 과적합이 의심

## 피쳐 중요도 구분하기


각 피쳐들의 정보 이득값을 계산한다

In [ ]:
from xgboost import plot_importance
fig, ax = plt.subplots(1, 2, figsize = (25, 10))
plot_importance(model_2, ax = ax[0])
plot_importance(model_4, ax = ax[1])
plt.show()

p62

**max_depth**에 따라서 feature importance의 값이 달라진다.


* 즉(이미 알고 있었겠지만), importance_plot의 값 자체는 큰 의미가 없다
* 또한 "importance_plot의 순서가 당뇨병을 진단하는 우선순위라고 판단할 수 없다"

***그렇지만*** importance_plot의 순서가 당뇨병 진단에 영향을 미치는 정도를 구분한다는 해석은 가능하다

--> 여러번 확인했을 때 항상 상위에 있다면 중요하다 이정도?

## 4.5.2.3 부분 의존성 플롯 그리기

이 방법은 model을 imput으로 넣지 않는다. 

즉, 부분 의존성 기법은 학습 데이터를 철저하게 분석해서 모델이 어떻게 학습할 것인지 예상하는 XAI기법이다


In [ ]:
from pdpbox import info_plots
from pdpbox import pdp
pima_data = dataset
pima_features = dataset.columns[:8]
pima_target = dataset.columns[8]

fig, ax, summary_df = info_plots.target_plot(
    df = pima_data,
    feature = 'Glucose',
    feature_name = 'Glucose',
    target = pima_target)

위 그림에서 X축은 GTT수치, 왼쪽Y는 각 구간별 데이터 갯수, 오른쪽Y는 당뇨병 진단여부
* 가장 왼쪽 막대 그래프는 GTT가 87미만인 경우 82개의 데이터가 존재하고, 이 그룹의 당뇨를 진단할 확률은 7.3%
* 가장 우측 막대는 GTT가 164~199인 환자들의 당뇨 진단 확률은 82.6% 이다

--> 기존 의학 지식과 관련하여 충분히 합리적인 결과가 나왔다

In [ ]:
fig, ax, summary_df = info_plots.target_plot(
    df = pima_data,
    feature = 'BloodPressure',
    feature_name = 'BloodPressure',
    target = pima_target)

* 80~120인 경우 고혈압이라고 할 수 있다. 고혈압의 당뇨 확률은 정상보다 10%가량 높다
* 하지만 가장 높은 확률이 46.2%로 혈압만으로 당뇨를 진단하기에는 무리가 있다.

--> 이제 XGB 모델과 비교를 통해, 데이터와 동일한 결과를 얻었는지 확인해보자

In [ ]:
fig, ax, summary_df = info_plots.actual_plot(
    model = model_2,
    X = pima_data[pima_features],
    feature = 'Glucose',
    feature_name = 'Glucose',
    show_percentile = True
)

fig, ax, summary_df = info_plots.target_plot(
    df = pima_data,
    feature = 'Glucose',
    feature_name = 'Glucose',
    target = pima_target)

* model_2
--> 통계량 만큼 잘 나오는 것 같다. 


* model_4
**--> 통계량보다 편향 되어 있는것 같다.**
글루코스가 144이상인 경우에만! 잘 작동하는것 같다.왜 그럴까?

In [ ]:
fig, ax, summary_df = info_plots.actual_plot(
    model = model_4,
    X = pima_data[pima_features],
    feature = 'BloodPressure',
    feature_name = 'BloodPressure',
    show_percentile = True
)

fig, ax, summary_df = info_plots.target_plot(
    df = pima_data,
    feature = 'BloodPressure',
    feature_name = 'BloodPressure',
    target = pima_target)

* model_2
--> 통계량과 어느정도 일치하는 모습을 보인다



* model_4
--> 혈압에 대한 정보를 반영하지 못하고 있다.


In [ ]:
# 이번에는 부분 의존 플랏을 그려본다
pdp_gc = pdp.pdp_isolate(
    model = model_2,
    dataset = pima_data,
    model_features = pima_features,
    feature = 'Glucose'
)

# plot정보 설정
fig, ax = pdp.pdp_plot(
    pdp_gc,
    'Glucose',
    plot_lines = False,
    frac_to_plot=0.5,
    plot_pts_dist = True
)

pdp_isolate는 'Glucose' 피쳐 하나에 대한 부분 의존성 수치를 반환한다.

* info_plots.target_plot에서 확인한 것과 같이 'Glucose'가 증가할 수록 당뇨병 확률이 증가한다

In [ ]:
# plot정보 설정
fig, ax = pdp.pdp_plot(
    pdp_gc,
    'BloodPressure',
    plot_lines = True,
    frac_to_plot=0.5,
    plot_pts_dist = True
)

In [ ]:
pdp_interaction = pdp.pdp_interact(
model = model_4
    ,dataset = pima_data
    ,model_features= pima_features
    ,features = ['BloodPressure', 'Glucose']
)

fig,ax = pdp.pdp_interact_plot(
    pdp_interact_out=pdp_interaction
    ,feature_names = ['BloodPressure', 'Glucose']
    , plot_type ='contour'
    , x_quantile = True
    , plot_pdp = True
)

* model_2

--> X축에 평행한 모습을 보인다. 즉, 혈압보다는 GTT에 더 큰 영향을 받고 있다.

* model_4

--> 굉장히 들쭉날쭉한 모습이다. 기존에 살펴본 통계량과 매치가 전혀되지 않는다

In [ ]:
# 이번에는 부분 의존 플랏을 그려본다
pdp_gc = pdp.pdp_isolate(
    model = model_4,
    dataset = pima_data,
    model_features = pima_features,
    feature = 'BloodPressure'
)

# plot정보 설정
fig, ax = pdp.pdp_plot(
    pdp_gc,
    'BloodPressure',
    plot_lines = False,
    frac_to_plot=0.5,
    plot_pts_dist = True
)

* model_2 & model_4

--> 혈압에 대해 공통적으로 당뇨병 진단에 음의 영향력을 보인다.

이것은 이전에 살펴본 통계량과 전혀 다른 결과를 보이는 것이다!


# 해석 결과가 상충한다면, 무엇이 옳은 해석일까?


* feature importance는 방향이 존재하지 않는다. (양, 음의 영향력인지)
* 피쳐의 scale에 따라 모델에 미치는 영향을 파악할 수 없다
* 피쳐간 의존성이 존재하는 경우 결과를 신뢰할 수 없다

--> 이런 한계를 개선하기 위해 pdp를 이용했었음

분명 혈압이 상승하면 당뇨병 진단에 도움이 되는 것으로 보임.<br>
하지만 지금까지 학습한 모델에서 혈압은 당뇨병 진단에 음의 영향을 보인다<br>

<br><br>
p78. <br>
우리가 가진 데이터의 최대치로는 '의학적인 고혈압'을 표시할 수 없다, <br>
모델은 정상 범주 혈압을 학습해서 당뇨 진단에 음의 영향력을 발휘 했다고 해석할 수 있다 

* 개인적인 생각


1. 본 데이터의 혈압은 확장기 혈압(120/80 에서 80에 해당)이므로, 값이 높을수록 고혈압이 될 가능성이 존재
2. 결국 모델은 혈압에 대한 정보를 반영하지 못했다
3. 그럼에도 불구하고 model_4에서 혈압을 제외하면 정확도는 낮아진다
4. logistic 모델을 적용하면 83%의 정확도로 가장 높은 결과를 얻을 수 있다 (model_2: 75%, model_4: 71%)

--> **pdp로 통계량을 확인한 다음 내가 만든 모델이 통계량을 쫓아가는지 확인해야 할 듯!**

In [ ]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(random_state=0).fit(x_train, y_train)
pred = clf.predict(x_test)

logReg_coeff = pd.DataFrame({'feature_name': pima_features, 'model_coefficient': clf.coef_.transpose().flatten()})
logReg_coeff

In [ ]:
predictions = [round(value) for value in pred]
acc = accuracy_score(y_test, predictions)
print(f"logistic is {acc*100}")

In [ ]:
X_re = dataset.loc[:, ['Pregnancies','Glucose', 'SkinThickness','Insulin','BMI','DiabetesPedigreeFunction','Age']]
y_re = dataset.iloc[:, 8]

x_train_re, x_test_re, y_train_re, y_test_re = train_test_split(X_re, y_re, test_size = 0.1, random_state = 7)

model_2_re = XGBClassifier(max_depth=2)
model_2_re.fit(x_train_re, y_train_re)

# 예측하기
y_pred = model_2_re.predict(x_test_re)
predictions = [round(value) for value in y_pred]
acc = accuracy_score(y_test_re, predictions)
print(f"model_2_re is {acc*100}")

# 예측하기
y_pred = model_2.predict(x_test)
predictions = [round(value) for value in y_pred]
acc = accuracy_score(y_test, predictions)
print(f"model_2 is {acc*100}")

In [ ]:
X_re = dataset.loc[:, ['Pregnancies','Glucose', 'SkinThickness','Insulin','BMI','DiabetesPedigreeFunction','Age']]
y_re = dataset.iloc[:, 8]

x_train_re, x_test_re, y_train_re, y_test_re = train_test_split(X_re, y_re, test_size = 0.1, random_state = 7)

model_4_re = XGBClassifier(max_depth=4)
model_4_re.fit(x_train_re, y_train_re)

# 예측하기
y_pred = model_4_re.predict(x_test_re)
predictions = [round(value) for value in y_pred]
acc = accuracy_score(y_test_re, predictions)
print(f"model_4_re is {acc*100}")

# 예측하기
y_pred = model_4.predict(x_test)
predictions = [round(value) for value in y_pred]
acc = accuracy_score(y_test, predictions)
print(f"model_4 is {acc*100}")

In [ ]:
plot_tree(model_4_re)